# Proxy Perplexity

The goal of this notebook is to unravel the feasibility of the usage of the perplexity metric as a proxy for the groundtruth metric dataset of 50 samples with genertions in the models Llama3.2-3B-Instruct using 5 different random generations.


In [2]:
import polars as pl
import seaborn as sns
import json
import os

### Importing Data

In [13]:
#### Import collections
collections_list = []
for dir in os.listdir("collections"):
    for file in os.listdir(f"collections/{dir}"):
        collections_list.append(pl.read_ipc(f"collections/{dir}/{file}").with_columns(pl.lit(dir).alias("seed")))
collections = pl.concat(collections_list)

    

In [12]:
collections.head(5)

collection_idx,test_idx,input,evaluation,seed
i64,i64,"array[i64, 100]",f64,str
1000,0,"[0, 1, … 0]",0.0,"""7270"""
1000,1,"[0, 1, … 0]",1.0,"""7270"""
1000,2,"[0, 1, … 0]",1.0,"""7270"""
1000,3,"[0, 1, … 0]",1.0,"""7270"""
1000,4,"[0, 1, … 0]",1.0,"""7270"""


In [15]:
#### Import pre-collections
pre_collections_list = []
for dir in os.listdir("pre_collections"):
    for file in os.listdir(f"pre_collections/{dir}"):
       pre_collections_list.append(pl.read_ipc(f"pre_collections/{dir}/{file}").with_columns(pl.lit(dir).alias("seed")))
pre_collections = pl.concat(pre_collections_list)

In [18]:
pre_collections.head(5)

collection_idx,test_idx,input,predicted_output,true_output,seed
i64,i64,"array[i64, 100]",str,list[str],str
1400,0,"[0, 0, … 1]","""Judith Keppel""","[""Judith Cynthia Aline Keppel""]","""7270"""
1400,1,"[0, 0, … 1]","""George W. Bush""","[""George W. Bush"", ""Bush""]","""7270"""
1400,2,"[0, 0, … 1]","""Sammi Smith""","[""Kris Kristofferson""]","""7270"""
1400,3,"[0, 0, … 1]","""October 27, 1904.""","[""October 27 , 1904"", ""1904""]","""7270"""
1400,4,"[0, 0, … 1]","""2004""","[""2004"", ""February 25 , 2004""]","""7270"""
